## Evaluation code

In [2]:
from io_ import readFiles
import numpy as np

# This is a sample test folder
test_folder = 'test_images'
image_paths = readFiles(test_folder)
print('Below are test images with their ground_truth class index appeneded:')
print(image_paths)

Below are test images with their ground_truth class index appeneded:
['a_0.jpg' 'b_21.jpg' 'c_7.JPG' 'd_4.jpg' 'e_22.jpg']


## Score each of the images based on the likelihood it belongs to a class. Larger scores indicate higher likelihood.

Classes are arranged alphabeticall, [1_Capitol, 1_Chijmes,..., 1_scis, americano, ... taco]. Screen-shot is attached.

The first thirteen belong to buidings and the next fifty nine belong to food, this makes for a total of 72 classes.

Scores should take the form of a [Nx72] array, where N is the number of test images.


In [3]:
################# Code for sorting the classes (not needed for project) ##################

# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import numpy as np

# train_dir = '../training'



# train_datagen = ImageDataGenerator() 
# train_generator = train_datagen.flow_from_directory(
#     train_dir) 

# a = train_generator.class_indices
# l = []
# for i in a:
#     l.append(i)
# np.save('sorted_classes.npy', l)

In [4]:
sorted_class = np.load('sorted_classes.npy')
print(sorted_class[0], ' is class ', 0)
print(sorted_class[10], ' is class ', 10)
print(sorted_class[21], ' is class ', 21)

1_Capitol  is class  0
1_library  is class  10
cafe_coffee_with_milk  is class  21


# Load Model


In [5]:
%%capture
# Your model will be tested as following
import tensorflow as tf

# Model reconstruction from JSON file
with open('goog_augment.json', 'r') as json_file:
    json_savedModel= json_file.read()

model = tf.keras.models.model_from_json(json_savedModel)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',
                   optimizer='adam',
                    metrics=['acc'])

# Load weights into the new model
model.load_weights('goog_augment.h5')

In [37]:
############## Write your test function here ##############


# Below is a possible mechanics for score computation;
# However, it must be modifed according to pre-processing amd network
test_folder = 'test_images'
image_paths = readFiles(test_folder)

from tensorflow.keras.preprocessing import image
num_images = len(image_paths)
scores = np.zeros([num_images, 72])
for i,im in enumerate(image_paths):
    img = image.load_img(test_folder + '/' + im, target_size=(224, 224))
    img = image.img_to_array(img)
    img = img.reshape([1,224,224,3])
    scores[i] = model.predict(img)
    

#     predictions = model.predict(img)
#     score = tf.nn.softmax(predictions[0])
    
#     print(
#     "This image most likely belongs to {} with a {:.2f} percent confidence."
#     .format(sorted_class[np.argmax(score)], 100 * np.max(score))
# )
    
#     percent = predictions[0]
#     sorted_a = np.argsort(-percent, axis=-1, kind='quicksort', order=None)
    
    
#     top5 = sorted_a[:5]
#     for x in top5:
#         print(x, np.round(100 * np.max(score[x]),2), sorted_class[x])
#     print()


In [38]:
print(scores.shape)

(5, 72)


## Demonstrating the evaluation functions on the first two test images

You need not modify code beyond this point.

In [28]:
from io_ import score
print('overall accuracy')

# simulated scores
scores = np.zeros([2,72])
scores[0,0] = 1000
scores[1,3] = 1000
scores[1,21] = 999

# Ground-truth: the first image belongs to class 0; the second image to class 13.
gt = np.array([0,21]) 

top1, top5 = score(scores, gt, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

overall accuracy
percentage top1 accuracy: 0.5
percentage top5 accuracy: 1.0


In [29]:
print('accuracy on building only')
mask = gt<13
top1, top5 = score(scores[mask,:13], gt[mask], 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

accuracy on building only
percentage top1 accuracy: 1.0
percentage top5 accuracy: 1.0


In [30]:
print('accuracy on food only')
mask = gt>=13
top1, top5 = score(scores[mask,13:], gt[mask]-13, 5)
print('percentage top1 accuracy:', top1)
print('percentage top5 accuracy:', top5)

accuracy on food only
percentage top1 accuracy: 1.0
percentage top5 accuracy: 1.0
